In [ ]:
## NEED TO INSTALL
# pip install isic-cli 

In [2]:
!isic collection list

If you have been granted special permissions, logging in with `isic user login` might return more data.

┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ ID  ┃ Name                      ┃ Public ┃ Pinned ┃ Locked ┃ DOI             ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 249 │ BCN20000                  │ True   │ False  │ False  │ None            │
│ 61  │ Challenge 2016: Test      │ True   │ True   │ True   │ None            │
│ 74  │ Challenge 2016: Training  │ True   │ True   │ True   │ None            │
│ 69  │ Challenge 2017: Test      │ True   │ True   │ True   │ None            │
│ 60  │ Challenge 2017: Training  │ True   │ True   │ True   │ None            │
│ 71  │ Challenge 2017:           │ True   │ True   │ True   │ None            │
│     │ Validation                │        │        │        │                 │
│ 64  │ Challenge 2018: Task 1-2: │ True   │ True   │ True   │ None            │
│   

In [7]:
!isic metadata download --collections 251

If you have been granted special permissions, logging in with `isic user login` might return more data.

isic_id,attribution,copyright_license,age_approx,anatom_site_general,benign_malignant,concomitant_biopsy,dermoscopic_type,diagnosis,diagnosis_confirm_type,family_hx_mm,fitzpatrick_skin_type,image_type,lesion_id,patient_id,personal_hx_mm,pixels_x,pixels_y,sex
ISIC_6230191,Hospital Italiano de Buenos Aires,CC-BY,25,anterior torso,malignant,True,contact polarized,melanoma,histopathology,,II,dermoscopic,IL_8124990,IP_8542863,True,640,480,male
ISIC_4485929,Hospital Italiano de Buenos Aires,CC-BY,65,anterior torso,benign,False,contact polarized,seborrheic keratosis,,,I,dermoscopic,IL_1921228,IP_9326183,True,2448,2448,female
ISIC_3079785,Hospital Italiano de Buenos Aires,CC-BY,45,lower extremity,benign,True,contact polarized,nevus,histopathology,False,II,dermoscopic,IL_3797557,IP_4906546,False,640,480,male
ISIC_9677008,Hospital Italiano de Buenos Aires,CC-BY,30,posterior torso,malignant,Tr

In [4]:
!pwd

/Users/endothermiic/Desktop/GitHub/APS360_Project


In [5]:
!isic image download --collections 251 TestImages/

If you have been granted special permissions, logging in with `isic user login` might return more data.


Successfully downloaded 1,616 images to TestImages/.
Successfully wrote 1,616 metadata records to TestImages/metadata.csv.
Successfully wrote attributions to TestImages/attribution.txt.
Successfully wrote 1 license(s) to TestImages/licenses.


In [6]:
# open metadata csv file
import csv
with open('metadata.csv','r') as csvfile:
    data = csv.reader(csvfile)

    meta = []
    for row in data:
        meta.append(row)

# visualize data cols
meta[0:3]

[['isic_id',
  'attribution',
  'copyright_license',
  'age_approx',
  'anatom_site_general',
  'benign_malignant',
  'concomitant_biopsy',
  'dermoscopic_type',
  'diagnosis',
  'diagnosis_confirm_type',
  'family_hx_mm',
  'fitzpatrick_skin_type',
  'image_type',
  'lesion_id',
  'patient_id',
  'personal_hx_mm',
  'pixels_x',
  'pixels_y',
  'sex'],
 ['ISIC_6230191',
  'Hospital Italiano de Buenos Aires',
  'CC-BY',
  '25',
  'anterior torso',
  'malignant',
  'True',
  'contact polarized',
  'melanoma',
  'histopathology',
  '',
  'II',
  'dermoscopic',
  'IL_8124990',
  'IP_8542863',
  'True',
  '640',
  '480',
  'male'],
 ['ISIC_4485929',
  'Hospital Italiano de Buenos Aires',
  'CC-BY',
  '65',
  'anterior torso',
  'benign',
  'False',
  'contact polarized',
  'seborrheic keratosis',
  '',
  '',
  'I',
  'dermoscopic',
  'IL_1921228',
  'IP_9326183',
  'True',
  '2448',
  '2448',
  'female']]

In [15]:
import numpy as np
meta = np.array(meta)

In [32]:
# extract image IDs (first column)
img_ids = meta[1:, 0]

In [33]:
img_ids

array(['ISIC_6230191', 'ISIC_4485929', 'ISIC_3079785', ...,
       'ISIC_9314666', 'ISIC_8229835', 'ISIC_7874486'], dtype='<U33')

In [47]:
img_types = meta[1:, 12]

In [48]:
img_types # only want dermoscopic images

array(['dermoscopic', 'dermoscopic', 'dermoscopic', ..., 'dermoscopic',
       'clinical: close-up', 'dermoscopic'], dtype='<U33')

In [55]:
img_diagnoses = meta[1:, 8]

In [56]:
img_diagnoses

array(['melanoma', 'seborrheic keratosis', 'nevus', ...,
       'actinic keratosis', 'basal cell carcinoma',
       'basal cell carcinoma'], dtype='<U33')

In [58]:
diagnosis = {'basal cell carcinoma': 'bcc', 'nevus':'nv', 'melanoma' : 'mel', 'seborrheic keratosis' : 'bkl', 'solar lentigo' : 'bkl'}

In [59]:
# create a new folder for data and navigate to this directory
import os
import shutil
import numpy as np

if not os.path.exists("TestDataset"):
  os.makedirs("TestDataset")

%cd TestDataset

# create a new folder for each class
classes = ['bkl', 'mel', 'nv', 'bcc']
for c in classes:
  os.makedirs(c, exist_ok=True)

In [71]:
# iterate over all images in working directory
for filename in os.listdir('TestImages'):
    filename = filename.removesuffix('.jpg')

    # use the extracted image IDs and match to a corresponding class
    if filename in img_ids:
        i = np.where(img_ids == filename)[0][0]
        if img_diagnoses[i] in diagnosis.keys() and img_types[i] == 'dermoscopic':
          class_name = diagnosis[img_diagnoses[i]]
         # move image to corresponding class folder
          shutil.move('TestImages' + '/' + filename + '.jpg',
                      os.path.join('TestDataset/' + class_name, filename + '.jpg'))

In [72]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler as srs
import random
import matplotlib.pyplot as plt
from collections import Counter
import torch.optim as optim
import time
from torch.utils.data import DataLoader


In [77]:
dir = '/Users/endothermiic/Desktop/GitHub/APS360_Project/TestDataset'
transform = transforms.Compose([transforms.ToTensor(),
         transforms.Resize((224, 224)), # images are either 450 * 600 or 1024 * 1024
                                # all resized to 224 * 224 for ResNet
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# transform input images to match requirements; load all images to all_data
test_data = torchvision.datasets.ImageFolder(dir, transform=transform)

In [78]:
test_data

Dataset ImageFolder
    Number of datapoints: 1032
    Root location: /Users/endothermiic/Desktop/GitHub/APS360_Project/TestDataset
    StandardTransform
Transform: Compose(
               ToTensor()
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )